In [1]:
from pyseter.sort import load_features, prep_images

root = '/Users/philtpatton/datasets/guiana/Guiana dolphins 24_25'
image_dir = root + '/IG'
# prep_images(root)

In [2]:
import numpy as np

feature_path = root + '/features/ig_features.npy'
feature_dict = np.load(feature_path, allow_pickle=True).item()

# unpack the dictionary into arrays
filenames = np.array(list(feature_dict.keys()))
feature_array = np.array(list(feature_dict.values()))

In [3]:
from sklearn.metrics.pairwise import cosine_distances
from sklearn.cluster import AgglomerativeClustering
from pyseter.sort import report_cluster_results, format_ids

# convert similarity threshold to distance
match_threshold = 0.75
distance_threshold = 1 - match_threshold

dist_matrix = 1 - cosine_distances(feature_array)
# cluster using average linkage
hac_average = AgglomerativeClustering(
    n_clusters=None, 
    distance_threshold=distance_threshold, 
    linkage='average',
    metric='precomputed'
).fit(dist_matrix)
labels_average = hac_average.labels_
report_cluster_results(labels_average)

Found 1190 clusters.
Largest cluster has 5443 images.


In [9]:
# single linkage is necessary when using cosine distance
distance_threshold = 0.5
hac_complete = AgglomerativeClustering(
    distance_threshold=distance_threshold, 
    n_clusters=None, 
    metric='cosine', 
    linkage='complete'
).fit(feature_array)
labels_complete = hac_complete.labels_
report_cluster_results(labels_complete)

Found 1276 clusters.
Largest cluster has 53 images.


In [7]:
from pyseter.grade import rate_distinctiveness
ers = rate_distinctiveness(feature_array, 0.75)

/Users/philtpatton/source/repos/pyseter/pyseter/grade.py:15: UserWarning: Distinctiveness grades are experimental and should be verified.
  warn('Distinctiveness grades are experimental and should be verified.')


Unrecognizable identity cluster consists of 1120 images.


In [80]:
import pandas as pd
import os

ers_df = pd.DataFrame(dict(image=filenames, ers=ers))

autosort_dir = root + '/IG_autosort'
file_id_map = {}
for path, _, files in os.walk(autosort_dir):
    for file in files:
        if '.DS_Store' in file:
            continue
        full_path = os.path.join(path, file)
        id_folder = full_path.split('/')[-3]
        file_id_map[file] = id_folder
       
ers_df['autosort_id'] = [file_id_map[f] for f in ers_df.image]
id_ers = ers_df.groupby('autosort_id')['ers'].median().rename('id_ers').reset_index()
ers_df = ers_df.merge(id_ers)[['autosort_id', 'id_ers', 'image', 'ers']].sort_values('autosort_id').reset_index(drop=True)

out_path = root + '/ig_ers.csv'
ers_df.to_csv(out_path, index=False)

# SE

In [81]:
image_dir = root + '/SE'
# prep_images(root)

feature_path = root + '/features/se_features.npy'
feature_dict = np.load(feature_path, allow_pickle=True).item()

# unpack the dictionary into arrays
filenames = np.array(list(feature_dict.keys()))
feature_array = np.array(list(feature_dict.values()))

ers = rate_distinctiveness(feature_array, 0.75)

ers_df = pd.DataFrame(dict(image=filenames, ers=ers))

autosort_dir = root + '/SE_autosort'
file_id_map = {}
for path, _, files in os.walk(autosort_dir):
    for file in files:
        if '.DS_Store' in file:
            continue
        full_path = os.path.join(path, file)
        id_folder = full_path.split('/')[-3]
        file_id_map[file] = id_folder
       
ers_df['autosort_id'] = [file_id_map[f] for f in ers_df.image]
id_ers = ers_df.groupby('autosort_id')['ers'].median().rename('id_ers').reset_index()
ers_df = ers_df.merge(id_ers)[['autosort_id', 'id_ers', 'image', 'ers']].sort_values('autosort_id').reset_index(drop=True)

out_path = root + '/se_ers.csv'
ers_df.to_csv(out_path, index=False)

/Users/philtpatton/source/repos/pyseter/pyseter/grade.py:15: UserWarning: Distinctiveness grades are experimental and should be verified.
  warn('Distinctiveness grades are experimental and should be verified.')


Unrecognizable identity cluster consists of 1279 images.
